In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score
import warnings

In [12]:
class CustomMNISTNet(nn.Module):
    def __init__(self):
        super(CustomMNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(3136, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [13]:
def load_data(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
    return train_loader, test_loader

In [14]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [8]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')
    return accuracy

In [9]:
def save_onnx(model, device):
    model.eval()
    dummy_input = torch.randn(1, 1, 28, 28).to(device)
    torch.onnx.export(model, dummy_input, "mnist_cnn.onnx", verbose=True)
    print("Model saved in ONNX format")

In [10]:
def main():
    # Check CUDA availability
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if not torch.cuda.is_available():
        print("CUDA is not available. Running on CPU.")
    else:
        print(f"Running on GPU: {torch.cuda.get_device_name(0)}")

    model = CustomMNISTNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    train_loader, test_loader = load_data()

    print(model)

    best_accuracy = 0
    for epoch in range(1, 11):
        train(model, device, train_loader, optimizer, criterion, epoch)
        accuracy = test(model, device, test_loader, criterion)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_model.pth')
    
    print(f"Best test accuracy: {best_accuracy:.2f}%")

    # Load the best model and save in ONNX format
    model.load_state_dict(torch.load('best_model.pth'))
    save_onnx(model, device)

In [11]:
if __name__ == '__main__':
    main()

Running on GPU: NVIDIA GeForce RTX 3050 Laptop GPU
CustomMNISTNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.327258
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.184445
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.029396
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.019887
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.019392
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.128936
Train Epoch: 1 [

In [18]:
def load_model(model_path):
    model = CustomMNISTNet()
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
    model.eval()
    return model

In [24]:
def evaluate_mnist_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    all_probs = []
    correct = 0
    total = 0
    inference_times = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            start_time = time.time()
            output = model(data)
            inference_time = time.time() - start_time
            inference_times.append(inference_time)

            probs = F.softmax(output, dim=1)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    accuracy = 100 * correct / total
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    all_probs = np.array(all_probs)

    # Confusion Matrix
    conf_matrix = confusion_matrix(all_targets, all_preds)

    # Precision, Recall, F1-Score
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted', zero_division=0)

    # ROC AUC Score (one-vs-all)
    roc_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr', average='weighted')

    # Top-3 Accuracy
    top3_preds = np.argsort(all_probs, axis=1)[:, -3:]
    top3_correct = np.any(top3_preds == all_targets.reshape(-1, 1), axis=1).sum()
    top3_accuracy = 100 * top3_correct / total

    # Inference Time
    avg_inference_time = sum(inference_times) / len(inference_times)

    # Model Size
    model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)

    results = {
        'Accuracy': accuracy,
        'Confusion Matrix': conf_matrix,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC AUC Score': roc_auc,
        'Top-3 Accuracy': top3_accuracy,
        'Average Inference Time (s)': avg_inference_time,
        'Model Size (parameters)': model_size
    }

    return results

# Usage
# Main execution
if __name__ == "__main__":
    model_path = 'best_model.pth'
    batch_size = 64
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the model
    model = load_model(model_path)
    model.to(device)

    # Load the data
    train_loader, test_loader = load_data(batch_size)

    # Evaluate the model
    results = evaluate_mnist_model(model, test_loader, device)
    
    # Print results
    for metric, value in results.items():
        print(f"{metric}:")
        print(value)
        print()

Accuracy:
99.43

Confusion Matrix:
[[ 977    0    0    0    0    0    2    1    0    0]
 [   0 1133    1    0    0    0    1    0    0    0]
 [   0    1 1029    0    1    0    0    1    0    0]
 [   0    0    1 1006    0    3    0    0    0    0]
 [   0    0    0    0  976    0    1    0    1    4]
 [   2    0    0    3    0  886    1    0    0    0]
 [   2    1    0    0    1    2  952    0    0    0]
 [   0    3    3    0    0    0    0 1017    1    4]
 [   2    0    2    0    0    0    0    0  968    2]
 [   0    0    0    0    4    3    0    2    1  999]]

Precision:
0.9943031525639734

Recall:
0.9943

F1-Score:
0.9942991837877173

ROC AUC Score:
0.9999728464430621

Top-3 Accuracy:
99.98

Average Inference Time (s):
0.0011664390563964843

Model Size (parameters):
477706

